In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dependencies to run the notebook

# !pip install torch==1.12.1
# !pip install torchmetrics==0.10.2
# !pip install torchvision==0.14.0
# !pip install texttable==1.6.4


<span style="color:darkviolet">
<font size="3">Download the below files from https://drive.google.com/drive/folders/1q50QMurzK9a5l4JBHWjf8VuWcZkbF7PM to run this notebook : <br>
1) train_bert_embeddings.pkl <br>
2) test_bert_embeddings.pkl <br> </font>
</span>


In [ ]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 6.0 MB/s 


In [ ]:
import pickle5 as pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.optim as optim
import warnings
import numpy as np
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import classification_report
from torch.utils.data.sampler import SubsetRandomSampler

warnings.filterwarnings("ignore")


In [ ]:
EMBEDDINGS_NAME = "xlnet"


In [ ]:
BASE_PATH = "/content/drive/MyDrive/CSCI-544 Group 18/nlp_data"
TRAIN_EMBEDDINGS = f"{BASE_PATH}/embeddings/train_{EMBEDDINGS_NAME}_embeddings.pkl"
TEST_EMBEDDINGS = f"{BASE_PATH}/embeddings/test_{EMBEDDINGS_NAME}_embeddings.pkl"
TRAIN_DATASET_PATH = f"{BASE_PATH}/legal_bert/data/tos_clauses_train.csv"
TEST_DATASET_PATH = f"{BASE_PATH}/legal_bert/data/tos_clauses_dev.csv"
RNN_MODEL_PATH = f"{BASE_PATH}/models/rnn_{EMBEDDINGS_NAME}_model.pt"
GRU_MODEL_PATH = f"{BASE_PATH}/models/gru_{EMBEDDINGS_NAME}_model.pt"
LSTM_MODEL_PATH = f"{BASE_PATH}/models/lstm_{EMBEDDINGS_NAME}_model.pt"
BILSTM_MODEL_PATH = f"{BASE_PATH}/models/bilstm_{EMBEDDINGS_NAME}_model.pt"


In [ ]:
# Read train BERT embeddings
with open(TRAIN_EMBEDDINGS, "rb") as f:
    training_data = pickle.load(f)


In [ ]:
# Read test BERT embeddings
with open(TEST_EMBEDDINGS, "rb") as f:
    testing_data = pickle.load(f)


In [ ]:
# check
for item in training_data:
    print("The data is : ", item)


The data is :  embeddings


In [ ]:
len(training_data["embeddings"]), training_data["embeddings"][0].shape


(7531, torch.Size([512, 768]))

<span style="color:darkviolet">
<font size="4">Get the labels from train and test files.</font>
</span>


In [ ]:
TRAIN_DATASET_PATH = f"{BASE_PATH}/tos_clauses_train.csv"
TEST_DATASET_PATH = f"{BASE_PATH}/tos_clauses_dev.csv"


In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, header=0)
test_df = pd.read_csv(TEST_DATASET_PATH, header=0)


In [ ]:
train_targets = train_df.label.values
test_targets = test_df.label.values


In [ ]:
test_targets


array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using Device: {device}")


Using Device: cuda


<span style="color:darkviolet">
<font size="4">Create Dataset, Train and Test Classes</font>
</span>


In [ ]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should
    override ``__len__``, that provides the size of the dataset,
    and ``__getitem__``, supporting integer indexing in range
    from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])


In [ ]:
class TOSDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.data1 = X
        self.data2 = Y
        self.transform = transform

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        x = self.data1[index]
        y = self.data2[index]

        if self.transform is not None:
            x = torch.tensor(x)

        return torch.squeeze(x, dim=1), torch.tensor(y)


In [ ]:
test_len = len(test_df)
train_len = len(train_df)
X_train_tensor = TOSDataset(train_df["sentences"], train_df["label"])
# X_test_tensor = Train_Model(test_df)

num_train = len(X_train_tensor)
indices = list(range(num_train))
np.random.shuffle(indices)
# split = int(np.floor(num_train))
# train_idx = indices[split:]

train_sampler = SubsetRandomSampler(indices)
# valid_sampler = SubsetRandomSampler(valid_idx)
print(train_sampler)
train_df_by_index = train_df.loc[indices]
# val_df_by_index = df_train.loc[valid_idx]
train_fair = sum(train_df_by_index["label"] == 0)
train_unfair = sum(train_df_by_index["label"] == 1)
# val_fair = sum(val_df_by_index['label'] == 0)
# val_unfair = sum(val_df_by_index['label'] == 1)

print("train_fair:" + str(train_fair))
print("train_unfair:" + str(train_unfair))
# print("val_fair:" + str(val_fair))
# print("val_unfair:" + str(val_unfair))


train_fair:6705
train_unfair:826


In [ ]:
train_data = TOSDataset(training_data["embeddings"], train_targets, transform=transforms.ToTensor())
test_data = TOSDataset(testing_data["embeddings"], test_targets, transform=transforms.ToTensor())


<span style="color:darkviolet">
<font size="4">Prepare Data loaders</font>
</span>


In [ ]:
# how many samples per batch to load
BATCH_SIZE = 20

# number of subprocesses to use for data loading
NUM_WORKERS = 0


In [ ]:
# prepare data loaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [ ]:
# check sizes
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print("Sample input size: ", sample_x.size())  # batch_size, seq_length
# print("Sample input: \n", sample_x)
print()
print("Sample label size: ", sample_y.size())  # batch_size
# print("Sample label: \n", sample_y)


Sample input size:  torch.Size([20, 512, 768])

Sample label size:  torch.Size([20])


In [ ]:
torch.squeeze(sample_x, dim=1).shape


torch.Size([20, 512, 768])

<span style="color:darkviolet">
<font size="5">SIMPLE RNN</font><br>
<font size="2.5">Number of hidden dimension : 20</font> <br>
<font size="2.5">Number of layers: 1</font> <br>
<font size="2.5">Number of epochs: 5</font> <br>
</span>


In [ ]:
class RNNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(RNNet, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=5, batch_first=True, nonlinearity="relu")

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(5, x.size(0), self.hidden_dim)).to(device)

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
# class RNNet(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):

#         super(RNNet, self).__init__()

#         # Number of hidden dimensions
#         self.hidden_dim = hidden_dim

#         # RNN
#         self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=2, batch_first=True, nonlinearity="relu")

#         self.fc_1 = nn.Linear(hidden_dim, 400)

#         self.dropout = nn.Dropout(p=0.33)

#         # Readout layer
#         self.out = nn.Linear(400, output_dim)

#     def forward(self, x):

#         # Initialize hidden state with zeros
#         h0 = Variable(torch.zeros(2, x.size(0), self.hidden_dim)).to(device)

#         # One time step
#         rnn_out, hn = self.rnn(x, h0)

#         fc_1_out = F.relu(self.fc_1(rnn_out[:, -1, :]))

#         fc_1_out = self.dropout(fc_1_out)

#         out = self.out(fc_1_out)

#         return out


In [ ]:
import time


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [ ]:
EMBEDDING_DIM = 768
OUTPUT_DIM = 2


In [ ]:
HIDDEN_DIM = 512
N_EPOCHS = 20


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight(
    "balanced", classes=np.unique(train_df_by_index["label"]), y=train_df_by_index["label"]
)
class_weight


array([0.56159582, 4.55871671])

In [ ]:
model = RNNet(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        # do gradient clipping
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0, norm_type=2)
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), RNN_MODEL_PATH)
        test_min_loss = test_loss


Epoch: 01 | Epoch Time: 0m 39s
	Training Loss: 0.035666 \Test Loss: 0.034037
Test loss decreased (inf --> 0.034037). Saving model...
Epoch: 02 | Epoch Time: 0m 38s
	Training Loss: 80048359382797482356375552.000000 \Test Loss: 204335091483009139649544192.000000
Epoch: 03 | Epoch Time: 0m 38s
	Training Loss: 202522704162449624743804928.000000 \Test Loss: 204335091483009139649544192.000000


In [ ]:
y_pred_list = []
y_targ_list = []
model = RNNet(EMBEDDING_DIM, RNN_HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load(RNN_MODEL_PATH))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))


              precision    recall  f1-score   support

           0       0.96      0.83      0.89      1677
           1       0.35      0.75      0.48       206

    accuracy                           0.82      1883
   macro avg       0.66      0.79      0.69      1883
weighted avg       0.90      0.82      0.85      1883



Accuracy of simple RNN : 0.7413701415061951
F1 score of simple RNN : 0.5635073184967041
Precision of simple RNN : 0.5633978843688965
Recall of simple RNN : 0.6227356195449829


<span style="color:darkviolet">
<font size="5">Gated RNN</font><br>
</span>


In [ ]:
class GRU_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(GRU_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
GRU_HIDDEN_DIM = 512
GRU_N_EPOCHS = 6


In [ ]:
model_gru = GRU_Network(EMBEDDING_DIM, GRU_HIDDEN_DIM, OUTPUT_DIM)
print(model_gru)


In [ ]:
def train_GRU(model, train_loader, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in train_loader:

        train = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()

        output = model(train)

        loss = criterion(output, labels)

        acc = binary_accuracy(output.argmax(-1), labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)


In [ ]:
N_EPOCHS = 5

best_valid_loss = float("inf")

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1 / train_fair, 1 / train_unfair]))
# criterion = nn.NLLLoss()

optimizer_gru = optim.Adam(model_gru.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    gru_train_loss, gru_train_acc = train_RNN(model_gru, train_loader, optimizer_gru, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(
        f"\tTrain Loss: \
    {gru_train_loss:.3f} | Train Acc: {gru_train_acc*100:.2f}%"
    )


In [ ]:
test_loader_predict = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=0)


def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted.cpu())
    return prediction_list


predictions_gru = predict(model_gru, test_loader_predict)

a_tensor = torch.IntTensor(predictions_gru)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
gru_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average="macro")  # checked if weighted can be used
gru_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average="macro", num_classes=2)
gru_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average="macro", num_classes=2)
gru_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of Gated RNN :", gru_test_acc)
print("F1 score of Gated RNN :", gru_f1_score)
print("Precision of Gated RNN :", gru_precision)
print("Recall of Gated RNN :", gru_recall)


In [ ]:
table = Texttable()
table.set_cols_dtype(["a", "f", "f", "f", "f", "f"])
table.set_precision(5)
table.add_rows(
    [
        ["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
        ["Gated RNN", gru_train_acc, gru_test_acc, gru_f1_score, gru_precision, gru_recall],
    ]
)
print(table.draw(), "\n")


<span style="color:darkviolet">
<font size="5">LSTM</font><br>
</span>


In [ ]:
# LSTM


class LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


In [ ]:
def train_LSTM(model, train_loader, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in train_loader:

        train = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()

        output = model(train)

        loss = criterion(output, labels)

        acc = binary_accuracy(output.argmax(-1), labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)


In [ ]:
model_lstm = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_lstm)


In [ ]:
N_EPOCHS = 5

best_valid_loss = float("inf")

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1 / train_fair, 1 / train_unfair]))

optimizer = optim.Adam(model_lstm.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    lstm_train_loss, lstm_train_acc = train_LSTM(model_lstm, train_loader, optimizer, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(
        f"\tTrain Loss: \
    {lstm_train_loss:.3f} | Train Acc: {lstm_train_acc*100:.2f}%"
    )


In [ ]:
test_loader_predict = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=NUM_WORKERS)


def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted.cpu())
    return prediction_list


predictions_lstm = predict(model_lstm, test_loader_predict)

a_tensor = torch.IntTensor(predictions_lstm)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
lstm_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average="macro")  # checked if weighted can be used
lstm_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average="macro", num_classes=2)
lstm_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average="macro", num_classes=2)
lstm_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of LSTM :", lstm_test_acc)
print("F1 score of LSTM :", lstm_f1_score)
print("Precision of LSTM :", lstm_precision)
print("Recall of of LSTM :", lstm_recall)


In [ ]:
table = Texttable()
table.set_cols_dtype(["a", "f", "f", "f", "f", "f"])
table.set_precision(5)
table.add_rows(
    [
        ["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
        ["LSTM", lstm_train_acc, lstm_test_acc, lstm_f1_score, lstm_precision, lstm_recall],
    ]
)
print(table.draw(), "\n")


<span style="color:darkviolet">
<font size="5">Bi-LSTM</font><br>
</span>


In [ ]:
# LSTM


class Bi_LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(Bi_LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim))

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


In [ ]:
def train_Bi_LSTM(model, train_loader, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in train_loader:

        train = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()

        output = model(train)

        loss = criterion(output, labels)

        acc = binary_accuracy(output.argmax(-1), labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)


In [ ]:
model_bi_lstm = Bi_LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_bi_lstm)


In [ ]:
N_EPOCHS = 5

best_valid_loss = float("inf")

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1 / train_fair, 1 / train_unfair]))

optimizer = optim.Adam(model_bi_lstm.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    bi_lstm_train_loss, bi_lstm_train_acc = train_Bi_LSTM(model_bi_lstm, train_loader, optimizer, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(
        f"\tTrain Loss: \
    {bi_lstm_train_loss:.3f} | Train Acc: {bi_lstm_train_acc*100:.2f}%"
    )


In [ ]:
test_loader_predict = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=NUM_WORKERS)


def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted.cpu())
    return prediction_list


predictions_bi_lstm = predict(model_bi_lstm, test_loader_predict)

a_tensor = torch.IntTensor(predictions_bi_lstm)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
bi_lstm_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average="macro")  # checked if weighted can be used
bi_lstm_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average="macro", num_classes=2)
bi_lstm_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average="macro", num_classes=2)
bi_lstm_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of LSTM :", bi_lstm_test_acc)
print("F1 score of LSTM :", bi_lstm_f1_score)
print("Precision of LSTM :", bi_lstm_precision)
print("Recall of of LSTM :", bi_lstm_recall)


In [ ]:
table = Texttable()
table.set_cols_dtype(["a", "f", "f", "f", "f", "f"])
table.set_precision(5)
table.add_rows(
    [
        ["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
        ["Bi-LSTM", bi_lstm_train_acc, bi_lstm_test_acc, bi_lstm_f1_score, bi_lstm_precision, bi_lstm_recall],
    ]
)
print(table.draw(), "\n")


In [ ]:
# compare all
table = Texttable()
table.set_cols_dtype(["a", "f", "f", "f", "f", "f"])
table.set_precision(5)
table.add_rows(
    [
        ["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
        ["Simple RNN", rnn_train_acc, rnn_test_acc, rnn_f1_score, rnn_precision, rnn_recall],
        ["Gated RNN", gru_train_acc, gru_test_acc, gru_f1_score, gru_precision, gru_recall],
        ["LSTM", lstm_train_acc, lstm_test_acc, lstm_f1_score, lstm_precision, lstm_recall],
        ["Bi-LSTM", bi_lstm_train_acc, bi_lstm_test_acc, bi_lstm_f1_score, bi_lstm_precision, bi_lstm_recall],
    ]
)
print(table.draw(), "\n")


<span style="color:darkviolet">
<font size="3">Best Performance : Bi-LSTM</font><br>
</span>
